In [9]:
import requests
import xml.etree.ElementTree as ET

# 1. 발급받은 API 키를 여기에 입력하세요
service_key = "50hjvXuloV4qFNdrIUOglZZ6RGV7uq7pvpP0oxT+EV57bvEGnWfvqbjL939z/yfj9ta/H2Cn382mGmHpm4wmcw=="

# 2. 새로운 API 요청 URL
url = "http://apis.data.go.kr/1471000/CsmtcsIngdCpntInfoService01/getCsmtcsIngdCpntInfoService01"

# 3. 요청에 필요한 파라미터 설정
# 'INGR_KOR_NAME'을 추가하여 특정 원료를 검색할 수 있습니다.
params = {
    "serviceKey": service_key,
    "pageNo": 1,
    "numOfRows": 10,
    # "INGR_KOR_NAME": "비타민" # 특정 원료를 검색하려면 이 주석을 해제하세요
}

try:
    # 4. GET 요청 보내기
    print("API 요청을 보냅니다...")
    response = requests.get(url, params=params)
    response.raise_for_status()

    # 5. XML 응답 파싱
    root = ET.fromstring(response.content)
    
    # 6. XML에서 데이터 추출하여 출력
    # XML 태그 이름이 대문자로 바뀌었으므로, 이에 맞게 수정합니다.
    items = root.findall('.//item')

    if items:
        print("--- 검색 결과 (XML) ---")
        for item in items:
            ingr_name_tag = item.find('INGR_KOR_NAME')
            ingr_eng_name_tag = item.find('INGR_ENG_NAME')
            cas_no_tag = item.find('CAS_NO')
            
            ingr_name = ingr_name_tag.text if ingr_name_tag is not None else "정보 없음"
            ingr_eng_name = ingr_eng_name_tag.text if ingr_eng_name_tag is not None else "정보 없음"
            cas_no = cas_no_tag.text if cas_no_tag is not None else "정보 없음"
            
            print(f"원료명: {ingr_name}")
            print(f"영문명: {ingr_eng_name}")
            print(f"CAS No: {cas_no}")
            print("-" * 20)
    else:
        print("검색 결과가 없습니다.")

except requests.exceptions.RequestException as e:
    print(f"요청 중 오류 발생: {e}")
except ET.ParseError as e:
    print(f"XML 파싱 오류: {e}")

API 요청을 보냅니다...
--- 검색 결과 (XML) ---
원료명: 가공소금
영문명: None
CAS No: None
--------------------
원료명: 가지열매추출물
영문명: Solanum Melongena (Eggplant) Fruit Extract
CAS No: 84012-19-1
--------------------
원료명: 구멍쇠미역추출물
영문명: Agarum Cribrosum Extract
CAS No: None
--------------------
원료명: 루핀아미노산
영문명: Lupine Amino Acids
CAS No: None
--------------------
원료명: 류신
영문명: Leucine
,61-90-5(L-)39-2(DL-)
--------------------
원료명: 류코노스톡/무발효여과물
영문명: Leuconostoc/Radish Root Ferment Filtrate
CAS No: None
--------------------
원료명: 백혈구추출물
영문명: Leukocyte Extract
CAS No: None
--------------------
원료명: 리나칸투스 콤무니스추출물
영문명: Rhinacanthus Communis Extract
CAS No: None
--------------------
원료명: 리날룰
영문명: Linalool
CAS No: 78-70-6
--------------------
원료명: 리날릴아세테이트
영문명: Linalyl Acetate
CAS No: 115-95-7
--------------------


In [12]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from bs4 import BeautifulSoup

# WebDriver 설정 및 초기화
# 더 이상 chromedriver 경로를 수동으로 지정하지 않아도 됩니다.
service = ChromeService()
driver = webdriver.Chrome(service=service)

# Hwahae 스킨케어 랭킹 페이지로 이동
url = "https://www.hwahae.co.kr/rankings?english_name=category&theme_id=4"
driver.get(url)
time.sleep(3) # 페이지 로딩을 위해 잠시 대기

# 무한 스크롤로 모든 제품 로드
print("페이지를 스크롤하며 모든 제품을 로딩 중입니다...")
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2) # 새로운 콘텐츠가 로드될 때까지 대기

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

print("모든 제품 로딩 완료.")
time.sleep(3)

# 로드된 HTML 가져와서 BeautifulSoup으로 파싱
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 모든 제품 상세 페이지 링크 추출
# /products/ 로 시작하는 모든 <a> 태그를 찾습니다.
product_links = soup.select('a[href^="/products/"]')
all_product_urls = set()

for link in product_links:
    full_url = "https://www.hwahae.co.kr" + link.get('href')
    all_product_urls.add(full_url)

# WebDriver 종료
driver.quit()

print(f"총 {len(all_product_urls)}개의 제품 URL을 수집했습니다.")

페이지를 스크롤하며 모든 제품을 로딩 중입니다...
모든 제품 로딩 완료.
총 0개의 제품 URL을 수집했습니다.


In [13]:
# 셀레니움 WebDriver 재시작
service = ChromeService()
driver = webdriver.Chrome(service=service)

print("제품 상세 페이지 정보를 수집합니다.")
product_data = []

# 수집한 URL 리스트를 순회하며 각 페이지의 데이터를 가져옵니다.
for url in list(all_product_urls):
    try:
        driver.get(url)
        time.sleep(2) # 페이지 로딩 대기

        # BeautifulSoup으로 HTML 파싱
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 제품명 추출 (예시)
        # 실제 웹페이지의 HTML 구조를 보고 정확한 클래스명을 찾아야 합니다.
        product_name_tag = soup.find('div', class_='ProductView-info__name')
        product_name = product_name_tag.text.strip() if product_name_tag else "제품명 정보 없음"
        
        # 성분 목록 추출 (예시)
        ingredient_list_div = soup.find('div', class_='IngredientList-list')
        ingredients = []
        if ingredient_list_div:
            ingredient_tags = ingredient_list_div.find_all('p', class_='name')
            ingredients = [ing.text.strip() for ing in ingredient_tags]

        data = {
            'product_name': product_name,
            'ingredients': ingredients,
            'url': url
        }
        product_data.append(data)
        print(f"'{product_name}' 데이터 수집 완료.")

    except Exception as e:
        print(f"URL: {url} 에서 데이터 수집 실패: {e}")

# WebDriver 종료
driver.quit()

print(f"\n총 {len(product_data)}개의 제품 데이터를 수집했습니다.")
print("수집된 데이터 예시:")
if product_data:
    print(product_data[0])

제품 상세 페이지 정보를 수집합니다.

총 0개의 제품 데이터를 수집했습니다.
수집된 데이터 예시:


In [14]:
import json

# 수집된 데이터를 JSON 파일로 저장
with open('cosmetic_data.json', 'w', encoding='utf-8') as f:
    json.dump(product_data, f, ensure_ascii=False, indent=4)
    
print("데이터가 cosmetic_data.json 파일에 성공적으로 저장되었습니다.")

데이터가 cosmetic_data.json 파일에 성공적으로 저장되었습니다.


In [32]:
import subprocess
import json
import time

# 여기에 가장 최근에 복사한 cURL 명령어를 통째로 붙여넣으세요.
# 이 명령어는 프록시 설정, 헤더, URL 등 모든 정보가 포함되어야 합니다.
CURL_COMMAND = """
curl 'https://gateway.hwahae.co.kr/v14/products?category_id=4&limit=20&offset=160&ordering=-reco_score' \
-H 'Host: gateway.hwahae.co.kr' \
-H 'Accept: */*' \
-H 'hwahae-device-scale: 3' \
-H 'hwahae-app-version: 460' \
-H 'Accept-Language: ko-KR,ko;q=0.9' \
-H 'hwahae-timestamp: 2025-09-21 21:28:08' \
-H 'hwahae-user-id: 1700661' \
-H 'hwahae-platform: ios' \
-H 'User-Agent: HwaHae/460 CFNetwork/3826.600.41 Darwin/24.6.0' \
-H 'Connection: keep-alive' \
-H 'hwahae-device-id: 68B9B0AB-0408-4DFA-99C9-981F5F765349' \
-H 'hwahae-signature: dcba262d73c6fc68d908a425b692804b24b880f52377017abbae63bf8739b565' \
--proxy http://localhost:9090
"""

def run_curl_command(command):
    """curl 명령어를 실행하고 JSON 응답을 반환합니다."""
    try:
        # 터미널 명령어 실행
        result = subprocess.run(command, shell=True, capture_output=True, text=True, check=True)
        return json.loads(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Curl 명령어 실행 오류: {e.stderr}")
        return None
    except json.JSONDecodeError:
        print("JSON 파싱 오류: 응답이 JSON 형식이 아닙니다.")
        return None

print("API를 통해 데이터를 수집 중입니다...")

data = run_curl_command(CURL_COMMAND)

if data and 'results' in data:
    all_products = data['results']
    print(f"\n총 {len(all_products)}개의 제품 데이터를 수집했습니다.")

    with open('hwahae_products_full.json', 'w', encoding='utf-8') as f:
        json.dump(all_products, f, ensure_ascii=False, indent=4)
        
    print("모든 데이터가 'hwahae_products_full.json' 파일에 저장되었습니다.")
else:
    print("데이터를 가져오는 데 실패했습니다.")

API를 통해 데이터를 수집 중입니다...
데이터를 가져오는 데 실패했습니다.


In [42]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import re
import sys

# 사용자가 제공한 올리브영 제품 페이지 URL
PRODUCT_URL = "https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000231891&dispCatNo=100000100010013&trackingCd=Cat100000100010013_MID&t_page=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EA%B4%80&t_click=%EC%8A%A4%ED%82%A8/%ED%86%A0%EB%84%88_%EC%A0%84%EC%B2%B4__%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_number=2"

# 1. 크롬 드라이버 자동 설치 및 브라우저 실행
try:
    service = ChromeService(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(PRODUCT_URL)
    time.sleep(3) # 페이지 로딩 대기
except Exception as e:
    print(f"드라이버 실행 중 오류가 발생했습니다: {e}", file=sys.stderr)
    sys.exit(1)

try:
    # 2. '구매 정보' 탭을 찾아 클릭합니다.
    # 탭의 텍스트가 '구매정보'인 <li> 태그를 찾아서 클릭합니다.
    # 올리브영 웹페이지의 HTML 구조는 변경될 수 있으므로, 텍스트로 찾는 방법이 가장 안전합니다.
    purchase_info_tab = driver.find_element(By.XPATH, "//li[a[text()='구매정보']]")
    purchase_info_tab.click()
    
    # 탭 내용이 로드될 때까지 잠시 기다립니다.
    time.sleep(2)

    # 3. 페이지의 HTML 소스를 가져와 BeautifulSoup으로 파싱합니다.
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')

    # 4. 제품명 추출 (이전과 동일)
    product_name_tag = soup.find('p', class_='prd_name')
    product_name = product_name_tag.text.strip() if product_name_tag else "제품명을 찾을 수 없습니다."
    print(f"제품명: {product_name}")

    # 5. 성분 리스트 추출 (새로운 위치에서)
    # '화장품법에 따라 기재해야 하는 모든 성분'이라는 텍스트를 포함하는 dt 태그를 찾습니다.
    # 이제 '구매정보' 탭의 내용이 로드되었으므로 이 항목을 찾을 수 있습니다.
    ingredient_dt_tag = soup.find('dt', string=lambda text: '모든 성분' in text if text else '')
    
    if ingredient_dt_tag:
        # 그 다음 형제 태그인 <dd>를 찾습니다.
        ingredients_dd_tag = ingredient_dt_tag.find_next_sibling('dd')
        
        if ingredients_dd_tag:
            ingredients_text = ingredients_dd_tag.text.strip()
            ingredients_list = re.sub(r'\s*\([^)]*\)\s*', '', ingredients_text).split(',')
            
            print("\n성분 리스트:")
            for ingredient in ingredients_list:
                ingredient_stripped = ingredient.strip()
                if ingredient_stripped:
                    print(f"- {ingredient_stripped}")
        else:
            print("성분 목록을 찾을 수 없습니다.")
    else:
        print("전성분 정보를 담고 있는 항목을 찾을 수 없습니다.")

except Exception as e:
    print(f"오류가 발생했습니다: {e}")

finally:
    driver.quit()

제품명: [9/21하루특가/9월 올영픽] 아누아 피디알엔 100 히알루론산 부스터 토너 250ml 기획세트 (+40ml)

성분 리스트:
- 정제수
- 부틸렌글라이콜
- 글리세린
- 펜틸렌글라이콜
- 프로판다이올
- 사탕수수추출물
- 자일리틸글루코사이드
- 돌콩추출물
- 말라카이트추출물
- 자일리톨
- 인도멀구슬나무잎추출물
- 백미꽃추출물
- 인도멀구슬나무꽃추출물
- 아이비고드열매추출물
- 접시꽃추출물
- 알로에베라꽃추출물
- 하이드롤라이즈드하이알루로닉애씨드
- 가지열매추출물
- 홀리바질잎추출물
- 참산호말추출물
- 울금뿌리추출물
- 1
- 2-헥산다이올
- 베타인
- 하이드록시아세토페논
- 판테놀
- 에틸헥실글리세린
- 안하이드로자일리톨
- 알란토인
- 소듐디엔에이
- 다이프로필렌글라이콜
- 옥탄다이올
- 포스파티딜콜린
- 토코페롤
- 하이알루로닉애씨드/하이드롤라이즈드콜라겐크로스폴리머
- C12-13알케스-9
- 다이포타슘포스페이트
- 아세틸글루타민
- 카퍼트라이펩타이드-1
- 셀룰로오스검
- 다이소듐이디티에이
- 잔탄검


In [43]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import re
import json
import sys

# 스크래핑을 시작할 올리브영 카테고리 페이지 URL
# '스킨/토너 전체' 페이지
BASE_URL = "https://www.oliveyoung.co.kr/store/main/getPlanShopList.do?dispCatNo=100000100010013"

# 1. 크롬 드라이버 자동 설치 및 브라우저 실행
try:
    service = ChromeService(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    driver.get(BASE_URL)
    time.sleep(3)
except Exception as e:
    print(f"드라이버 실행 중 오류가 발생했습니다: {e}", file=sys.stderr)
    sys.exit(1)

all_product_urls = []
processed_urls = set()
all_products_data = []

print("📍 제품 목록 URL을 수집하고 있습니다. 잠시만 기다려 주세요...")

try:
    while True:
        # 현재 페이지의 HTML을 가져옵니다.
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # 제품 리스트에서 URL을 찾습니다.
        product_list = soup.find_all('div', class_='prd_info_area')
        
        for product in product_list:
            link_tag = product.find('a', href=True)
            if link_tag:
                url = "https://www.oliveyoung.co.kr" + link_tag['href']
                if url not in processed_urls:
                    all_product_urls.append(url)
                    processed_urls.add(url)

        # 다음 페이지 버튼을 찾습니다.
        next_button = driver.find_elements(By.XPATH, "//a[contains(text(), '다음')]")
        if next_button and 'on' in next_button[0].get_attribute('class'):
            next_button[0].click()
            time.sleep(3) # 다음 페이지 로딩 대기
        else:
            break # 다음 페이지가 없으면 루프 종료
    
    print(f"✅ 총 {len(all_product_urls)}개의 제품 URL을 수집했습니다.")
    print("⏳ 이제 각 제품 페이지에서 데이터를 수집합니다...")

    for i, url in enumerate(all_product_urls):
        print(f"➡️ {i+1}/{len(all_product_urls)} - {url} 스크래핑 중...")
        try:
            driver.get(url)
            time.sleep(2) # 페이지 로딩 대기

            # '구매 정보' 탭을 찾아 클릭합니다.
            purchase_info_tab = driver.find_element(By.XPATH, "//li[a[text()='구매정보']]")
            purchase_info_tab.click()
            time.sleep(1)
            
            # BeautifulSoup으로 HTML을 다시 파싱합니다.
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # 제품명 추출
            product_name_tag = soup.find('p', class_='prd_name')
            product_name = product_name_tag.text.strip() if product_name_tag else "제품명을 찾을 수 없습니다."

            # 성분 리스트 추출
            ingredient_dt_tag = soup.find('dt', string=lambda text: '모든 성분' in text if text else '')
            ingredients_list = []
            if ingredient_dt_tag:
                ingredients_dd_tag = ingredient_dt_tag.find_next_sibling('dd')
                if ingredients_dd_tag:
                    ingredients_text = ingredients_dd_tag.text.strip()
                    ingredients_list = [re.sub(r'\s*\([^)]*\)\s*', '', i).strip() for i in ingredients_text.split(',')]

            product_data = {
                "제품명": product_name,
                "URL": url,
                "성분리스트": ingredients_list
            }
            all_products_data.append(product_data)
        
        except Exception as e:
            print(f"❌ {url} 페이지 스크래핑 중 오류가 발생했습니다: {e}", file=sys.stderr)
            continue # 오류 발생 시 다음 URL로 넘어감

    # 수집한 데이터를 JSON 파일로 저장합니다.
    with open('oliveyoung_products.json', 'w', encoding='utf-8') as f:
        json.dump(all_products_data, f, ensure_ascii=False, indent=4)
        
    print("\n🎉 모든 데이터 수집이 완료되었습니다!")
    print(f"📦 총 {len(all_products_data)}개의 제품 데이터가 'oliveyoung_products.json' 파일에 저장되었습니다.")

except Exception as e:
    print(f"치명적인 오류가 발생했습니다: {e}", file=sys.stderr)

finally:
    driver.quit()

📍 제품 목록 URL을 수집하고 있습니다. 잠시만 기다려 주세요...
✅ 총 0개의 제품 URL을 수집했습니다.
⏳ 이제 각 제품 페이지에서 데이터를 수집합니다...

🎉 모든 데이터 수집이 완료되었습니다!
📦 총 0개의 제품 데이터가 'oliveyoung_products.json' 파일에 저장되었습니다.


In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import re
import json
import sys

# ************************************************
# 🎯 [핵심 설정] 수집할 최대 제품 개수를 여기에 입력하세요.
MAX_PRODUCTS_TO_SCRAPE = 10000  # 안전을 위해 20개로 설정. 이 숫자를 바꾸면 됩니다.
# ************************************************

# 수집할 카테고리 목록과 dispCatNo (카테고리 번호) 정의
CATEGORIES = {
    "스킨_토너": "100000100010013",
    "에센스_세럼_앰플": "100000100010014",
    "크림": "100000100010015",
    "로션": "100000100010016",
    "미스트_오일": "100000100010010"
}

# 1. 크롬 드라이버 설정 및 브라우저 실행
try:
    service = ChromeService(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
except Exception as e:
    print(f"드라이버 실행 중 오류가 발생했습니다: {e}", file=sys.stderr)
    sys.exit(1)

BASE_DOMAIN = "https://www.oliveyoung.co.kr"

# ************************************************************
# 2. 카테고리별로 루프를 돌며 데이터 수집을 반복합니다.
# ************************************************************
try:
    for category_name, cat_no in CATEGORIES.items():
        
        BASE_URL = f"{BASE_DOMAIN}/store/display/getMCategoryList.do?dispCatNo={cat_no}"
        all_product_urls = []
        processed_urls = set()
        all_products_data = []

        print(f"\n==================================================================")
        print(f"📦 [카테고리 시작] : {category_name} ({cat_no})")
        print(f"==================================================================")
        
        try:
            driver.get(BASE_URL)
            time.sleep(5) # 페이지 로딩 대기

            print(f"📍 제품 목록 URL을 수집하고 있습니다. 최대 {MAX_PRODUCTS_TO_SCRAPE}개까지 수집합니다...")
            
            # 2-1. URL 수집 루프 (페이지네이션 및 개수 제한 적용)
            while True:
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                
                # 'getGoodsDetail.do' 패턴을 포함하는 모든 <a> 태그를 찾습니다.
                link_tags = soup.find_all('a', href=lambda href: href and 'getGoodsDetail.do' in href)
                
                for link_tag in link_tags:
                    href = link_tag['href']
                    
                    # *********** https://xzio.tistory.com/72 ***********
                    if href.startswith(BASE_DOMAIN):
                        url = href # 이미 완전한 URL이면 그대로 사용
                    else:
                        url = BASE_DOMAIN + href # 상대 경로일 경우 도메인 추가

                    if url not in processed_urls:
                        all_product_urls.append(url)
                        processed_urls.add(url)
                        if len(all_product_urls) >= MAX_PRODUCTS_TO_SCRAPE:
                            break # URL 목표 개수 달성

                if len(all_product_urls) >= MAX_PRODUCTS_TO_SCRAPE:
                    break

                # 다음 페이지 버튼을 찾습니다.
                next_buttons = driver.find_elements(By.XPATH, "//a[contains(text(), '다음')]")
                if next_buttons:
                    next_button = next_buttons[0] 
                    if 'on' in next_button.get_attribute('class'):
                        next_button.click()
                        time.sleep(3) 
                    else:
                        break
                else:
                    break
            
            products_to_scrape = all_product_urls[:MAX_PRODUCTS_TO_SCRAPE]
            
            if not products_to_scrape:
                print("❌ 수집할 URL을 찾지 못했습니다. 다음 카테고리로 넘어갑니다.")
                continue

            print(f"✅ 총 {len(products_to_scrape)}개의 제품 URL 수집 완료.")
            print("⏳ 이제 각 제품 페이지에서 데이터를 수집합니다...")

            # 2-2. URL 리스트를 순회하며 데이터 스크래핑
            for i, url in enumerate(products_to_scrape):
                print(f"  ➡️ {i+1}/{len(products_to_scrape)} - 스크래핑 중...")
                try:
                    driver.get(url)
                    time.sleep(2) 

                    # '구매 정보' 탭을 찾아 클릭
                    purchase_info_tab = driver.find_element(By.XPATH, "//li[a[text()='구매정보']]")
                    purchase_info_tab.click()
                    time.sleep(1)
                    
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    
                    # 제품명 추출
                    product_name_tag = soup.find('p', class_='prd_name')
                    product_name = product_name_tag.text.strip() if product_name_tag else "제품명을 찾을 수 없습니다."

                    # 성분 리스트 추출
                    ingredient_dt_tag = soup.find('dt', string=lambda text: '모든 성분' in text if text else '')
                    ingredients_list = []
                    if ingredient_dt_tag:
                        ingredients_dd_tag = ingredient_dt_tag.find_next_sibling('dd')
                        if ingredients_dd_tag:
                            ingredients_text = ingredients_dd_tag.text.strip()
                            ingredients_list = [re.sub(r'\s*\([^)]*\)\s*', '', i).strip() for i in ingredients_text.split(',')]

                    product_data = {
                        "제품명": product_name,
                        "URL": url,
                        "성분리스트": ingredients_list
                    }
                    all_products_data.append(product_data)
                
                except Exception as e:
                    print(f"  ❌ {url} 페이지 스크래핑 중 오류가 발생했습니다. 건너뜁니다.", file=sys.stderr)
                    continue

            # 3. 수집한 데이터를 카테고리 이름으로 JSON 파일에 저장
            filename = f"oliveyoung_{category_name}_limited.json"
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(all_products_data, f, ensure_ascii=False, indent=4)
                
            print(f"🎉 {category_name} 데이터 수집 완료!")
            print(f"📦 총 {len(all_products_data)}개의 제품 데이터가 '{filename}' 파일에 저장되었습니다.")

        except Exception as e:
            print(f"⚠️ {category_name} 카테고리 처리 중 치명적인 오류가 발생했습니다: {e}", file=sys.stderr)
            continue 

finally:
    driver.quit()
    print("\n\n✅ 모든 카테고리 작업이 완료되었습니다. 브라우저가 종료됩니다.")


📦 [카테고리 시작] : 스킨_토너 (100000100010013)
📍 제품 목록 URL을 수집하고 있습니다. 최대 10000개까지 수집합니다...
✅ 총 24개의 제품 URL 수집 완료.
⏳ 이제 각 제품 페이지에서 데이터를 수집합니다...
  ➡️ 1/24 - 스크래핑 중...
  ➡️ 2/24 - 스크래핑 중...
  ➡️ 3/24 - 스크래핑 중...
  ➡️ 4/24 - 스크래핑 중...
  ➡️ 5/24 - 스크래핑 중...
  ➡️ 6/24 - 스크래핑 중...
  ➡️ 7/24 - 스크래핑 중...
  ➡️ 8/24 - 스크래핑 중...
  ➡️ 9/24 - 스크래핑 중...
  ➡️ 10/24 - 스크래핑 중...
  ➡️ 11/24 - 스크래핑 중...
  ➡️ 12/24 - 스크래핑 중...
  ➡️ 13/24 - 스크래핑 중...
  ➡️ 14/24 - 스크래핑 중...
  ➡️ 15/24 - 스크래핑 중...
  ➡️ 16/24 - 스크래핑 중...
  ➡️ 17/24 - 스크래핑 중...


✅ 모든 카테고리 작업이 완료되었습니다. 브라우저가 종료됩니다.


KeyboardInterrupt: 

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import re
import json
import sys

# ************************************************
# 🎯 [핵심 설정 1] 수집할 최대 제품 개수를 여기에 입력하세요.
MAX_PRODUCTS_TO_SCRAPE = 10000  
# 🎯 [핵심 설정 2] 카테고리당 탐색할 최대 페이지 수 (안전을 위해 설정)
MAX_PAGES_TO_EXPLORE = 30 # 5페이지까지만 탐색하도록 설정했습니다.
# ************************************************

# 수집할 카테고리 목록과 dispCatNo 정의
CATEGORIES = {
    "스킨_토너": "100000100010013",
    "에센스_세럼_앰플": "100000100010014",
    "크림": "100000100010015",
    "로션": "100000100010016",
    "미스트_오일": "100000100010010"
}

# 1. 크롬 드라이버 설정 및 브라우저 실행
try:
    service = ChromeService(executable_path=ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
except Exception as e:
    print(f"드라이버 실행 중 오류가 발생했습니다: {e}", file=sys.stderr)
    sys.exit(1)

BASE_DOMAIN = "https://www.oliveyoung.co.kr"

# ************************************************************
# 2. 카테고리별로 루프를 돌며 데이터 수집을 반복합니다.
# ************************************************************
try:
    for category_name, cat_no in CATEGORIES.items():
        
        # 기본 URL 구조 (pageIdx=1, rowsPerPage=24 설정)
        BASE_URL_TEMPLATE = f"{BASE_DOMAIN}/store/display/getMCategoryList.do?dispCatNo={cat_no}&prdSort=01&rowsPerPage=24"
        all_product_urls = []
        processed_urls = set()
        all_products_data = []

        print(f"\n==================================================================")
        print(f"📦 [카테고리 시작] : {category_name} ({cat_no})")
        print(f"==================================================================")
        
        try:
            print(f"📍 제품 목록 URL을 수집하고 있습니다. 최대 {MAX_PRODUCTS_TO_SCRAPE}개, 최대 {MAX_PAGES_TO_EXPLORE} 페이지까지 탐색합니다...")
            
            # 2-1. URL 수집 루프 (URL 파라미터 직접 변경 방식)
            for page_idx in range(1, MAX_PAGES_TO_EXPLORE + 1):
                
                # 페이지 번호가 삽입된 새로운 URL 생성
                page_url = f"{BASE_URL_TEMPLATE}&pageIdx={page_idx}"
                driver.get(page_url)
                time.sleep(3) # 페이지 로딩 대기
                
                print(f"   탐색 중: Page {page_idx} / 수집된 URL: {len(all_product_urls)}개")

                soup = BeautifulSoup(driver.page_source, 'html.parser')
                
                # 'getGoodsDetail.do' 패턴을 포함하는 모든 <a> 태그를 찾습니다.
                link_tags = soup.find_all('a', href=lambda href: href and 'getGoodsDetail.do' in href)
                
                if not link_tags and page_idx > 1:
                    # 2페이지 이상에서 더 이상 상품 링크가 없으면 마지막 페이지로 간주하고 종료
                    print(f"   [탐색 종료] Page {page_idx}에서 더 이상 상품을 찾을 수 없습니다.")
                    break
                    
                for link_tag in link_tags:
                    href = link_tag['href']
                    
                    # URL 중복 방지 로직 (이전 수정사항)
                    if href.startswith(BASE_DOMAIN):
                        url = href
                    else:
                        url = BASE_DOMAIN + href

                    if url not in processed_urls:
                        all_product_urls.append(url)
                        processed_urls.add(url)
                        
                        if len(all_product_urls) >= MAX_PRODUCTS_TO_SCRAPE:
                            break # URL 목표 개수 달성

                if len(all_product_urls) >= MAX_PRODUCTS_TO_SCRAPE:
                    break # 카테고리 URL 수집 루프 종료
            
            products_to_scrape = all_product_urls[:MAX_PRODUCTS_TO_SCRAPE]
            
            if not products_to_scrape:
                print("❌ 수집할 URL을 찾지 못했습니다. 다음 카테고리로 넘어갑니다.")
                continue

            print(f"✅ 총 {len(products_to_scrape)}개의 제품 URL 수집 완료.")
            print("⏳ 이제 각 제품 페이지에서 데이터를 수집합니다...")

            # 2-2. URL 리스트를 순회하며 데이터 스크래핑 (데이터 추출 로직은 동일)
            for i, url in enumerate(products_to_scrape):
                print(f"  ➡️ {i+1}/{len(products_to_scrape)} - 스크래핑 중...")
                try:
                    driver.get(url)
                    time.sleep(2) 

                    # '구매 정보' 탭을 찾아 클릭
                    purchase_info_tab = driver.find_element(By.XPATH, "//li[a[text()='구매정보']]")
                    purchase_info_tab.click()
                    time.sleep(1)
                    
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    
                    # 제품명 추출
                    product_name_tag = soup.find('p', class_='prd_name')
                    product_name = product_name_tag.text.strip() if product_name_tag else "제품명을 찾을 수 없습니다."

                    # 성분 리스트 추출
                    ingredient_dt_tag = soup.find('dt', string=lambda text: '모든 성분' in text if text else '')
                    ingredients_list = []
                    if ingredient_dt_tag:
                        ingredients_dd_tag = ingredient_dt_tag.find_next_sibling('dd')
                        if ingredients_dd_tag:
                            # 콤마로 분리되지 않은 성분 처리 및 불순물 제거는 후처리 단계(데이터 정제)에서 진행 예정
                            ingredients_text = ingredients_dd_tag.text.strip()
                            # 현재는 콤마 기준으로만 분리하여 최대한 원본 데이터를 가져옵니다.
                            ingredients_list = [i.strip() for i in ingredients_text.split(',')]

                    product_data = {
                        "제품명": product_name,
                        "URL": url,
                        "성분리스트": ingredients_list
                    }
                    all_products_data.append(product_data)
                
                except Exception as e:
                    print(f"  ❌ {url} 페이지 스크래핑 중 오류가 발생했습니다. 건너뜁니다.", file=sys.stderr)
                    continue

            # 3. 수집한 데이터를 카테고리 이름으로 JSON 파일에 저장
            filename = f"oliveyoung_{category_name}_raw_limited.json"
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(all_products_data, f, ensure_ascii=False, indent=4)
                
            print(f"🎉 {category_name} 데이터 수집 완료!")
            print(f"📦 총 {len(all_products_data)}개의 제품 데이터가 '{filename}' 파일에 저장되었습니다.")

        except Exception as e:
            print(f"⚠️ {category_name} 카테고리 처리 중 치명적인 오류가 발생했습니다: {e}", file=sys.stderr)
            continue 

finally:
    driver.quit()
    print("\n\n✅ 모든 카테고리 작업이 완료되었습니다. 브라우저가 종료됩니다.")


📦 [카테고리 시작] : 스킨_토너 (100000100010013)
📍 제품 목록 URL을 수집하고 있습니다. 최대 10000개, 최대 30 페이지까지 탐색합니다...
   탐색 중: Page 1 / 수집된 URL: 0개
   탐색 중: Page 2 / 수집된 URL: 24개
   탐색 중: Page 3 / 수집된 URL: 48개
   탐색 중: Page 4 / 수집된 URL: 72개
   탐색 중: Page 5 / 수집된 URL: 96개
   탐색 중: Page 6 / 수집된 URL: 120개
   탐색 중: Page 7 / 수집된 URL: 144개
   탐색 중: Page 8 / 수집된 URL: 168개
   탐색 중: Page 9 / 수집된 URL: 192개
   탐색 중: Page 10 / 수집된 URL: 216개
   탐색 중: Page 11 / 수집된 URL: 240개
   탐색 중: Page 12 / 수집된 URL: 264개
   탐색 중: Page 13 / 수집된 URL: 288개
   탐색 중: Page 14 / 수집된 URL: 312개
   탐색 중: Page 15 / 수집된 URL: 336개
   탐색 중: Page 16 / 수집된 URL: 360개
   탐색 중: Page 17 / 수집된 URL: 384개
   탐색 중: Page 18 / 수집된 URL: 386개
   [탐색 종료] Page 18에서 더 이상 상품을 찾을 수 없습니다.
✅ 총 386개의 제품 URL 수집 완료.
⏳ 이제 각 제품 페이지에서 데이터를 수집합니다...
  ➡️ 1/386 - 스크래핑 중...
  ➡️ 2/386 - 스크래핑 중...
  ➡️ 3/386 - 스크래핑 중...
  ➡️ 4/386 - 스크래핑 중...
  ➡️ 5/386 - 스크래핑 중...
  ➡️ 6/386 - 스크래핑 중...
  ➡️ 7/386 - 스크래핑 중...
  ➡️ 8/386 - 스크래핑 중...
  ➡️ 9/386 - 스크래핑 중...
  ➡️ 10/386 - 스크래핑 중...
